In [19]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [20]:
train_data = pd.read_csv('df_cleaned.csv')
df=train_data.copy()
df.head()

,floor_area,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,...,facility_type_Commercial_Other,facility_type_Education_College_or_university,facility_type_Education_Other_classroom,facility_type_Lodging_Hotel,facility_type_Multifamily_Uncategorized,facility_type_Office_Uncategorized,facility_type_Retail_Uncategorized,facility_type_Warehouse_Nonrefrigerated,facility_type_other,site_eui
0,-0.424272,-2.244014,-0.611748,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,248.682615
1,0.437535,-0.780906,-0.621640,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26.500150
2,0.461940,1.456789,-0.621640,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,24.693619
3,-0.448240,-0.737873,-0.621640,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,48.406926
4,-0.404999,1.585887,-0.611748,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.899395


In [21]:
df_y=df['site_eui']
df_x=df.drop('site_eui',axis=1)

In [22]:
# Model 1: fit a linear model with regularization
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
lasso_reg=Lasso()
lasso_reg.fit(df_x,df_y)
eui_lasso_prediction=lasso_reg.predict(df_x)
lasso_train_mse=mean_squared_error(df_y,eui_lasso_prediction)
lasso_train_rmse=np.sqrt(lasso_train_mse)
print('Lasso Train Scores:', lasso_train_rmse)

Lasso Train Scores: 52.52557581759984


In [23]:
scores_lasso=cross_val_score(lasso_reg,df_x,df_y,
                            scoring='neg_mean_squared_error',cv=10)
lasso_rmse_scores=np.sqrt(-scores_lasso)
print('Lasso Scores:', lasso_rmse_scores)
print('Lasso Mean Scores:', lasso_rmse_scores.mean())
print('Lasso alpha:', lasso_reg.alpha)
# the lasso model performs bad on training and generalize poorly- underfitting

Lasso Scores: [57.7051759  59.35667489 56.05976526 54.96763314 49.58184485 44.37959687
 38.07968197 46.03772084 60.050258   67.0786742 ]
Lasso Mean Scores: 53.32970259248132
Lasso alpha: 1.0


In [24]:
# Model 2: a decision tree model to capture nonlinear relationship
from sklearn.tree import DecisionTreeRegressor
tree_reg=DecisionTreeRegressor()
tree_reg.fit(df_x,df_y)
eui_tree_prediction=tree_reg.predict(df_x)
tree_train_mse=mean_squared_error(df_y,eui_tree_prediction)
tree_train_rmse=np.sqrt(tree_train_mse)
print('Tree Train Scores:', tree_train_rmse)
#tree model overfitting 

Tree Train Scores: 0.0


In [25]:
scores_tree=cross_val_score(tree_reg,df_x,df_y,
                            scoring='neg_mean_squared_error',cv=10)
tree_rmse_scores=np.sqrt(-scores_tree)
print('Tree Scores:', tree_rmse_scores)
print('Tree Mean Scores:', tree_rmse_scores.mean())
# the decision tree is overfiting

Tree Scores: [165.01333929  97.53640436  91.78431897  64.11110206  67.68712711
  52.46496194  54.32309878  59.80685545  81.59450394 160.4954465 ]
Tree Mean Scores: 89.48171583974408


In [26]:
# Model 3: a random forest regression model
from sklearn.ensemble import RandomForestRegressor
rf_reg=RandomForestRegressor(n_estimators=20, max_leaf_nodes=10,random_state=0)
rf_reg.fit(df_x,df_y)
eui_rf_prediction=rf_reg.predict(df_x)
rf_train_mse=mean_squared_error(df_y,eui_rf_prediction)
rf_train_rmse=np.sqrt(rf_train_mse)
print('RF Train Scores:', rf_train_rmse)
for name, score in sorted(zip(rf_reg.feature_importances_,df_x.columns),reverse=True):
    print(name,score)

RF Train Scores: 51.61024087649354
0.7654313287170433 energy_star_rating
0.08111280183540595 facility_type_other
0.03323158311982667 january_min_temp
0.030909576073477878 january_avg_temp
0.026023826221855183 building_class_Residential
0.01792972259980182 floor_area
0.016509085735580627 State_Factor_State_4
0.012438705658617695 building_class_Commercial
0.00536235150027953 february_avg_temp
0.0035471629308048676 days_below_20F
0.003536824859200563 heating_degree_days
0.002253017287647119 september_max_temp
0.0017140134604588509 precipitation_inches
0.0 year_built_old
0.0 year_built_new
0.0 snowfall_inches
0.0 snowdepth_inches
0.0 september_min_temp
0.0 september_avg_temp
0.0 october_min_temp
0.0 october_max_temp
0.0 october_avg_temp
0.0 november_min_temp
0.0 november_max_temp
0.0 november_avg_temp
0.0 may_min_temp
0.0 may_max_temp
0.0 may_avg_temp
0.0 max_wind_speed
0.0 march_min_temp
0.0 march_max_temp
0.0 march_avg_temp
0.0 level_0
0.0 june_min_temp
0.0 june_max_temp
0.0 june_avg_tem

In [28]:
# Model 3: a random forest regression model

rf_reg=RandomForestRegressor(n_estimators=20, max_leaf_nodes=10,random_state=0)
scores_rf = cross_val_score(
    rf_reg, df_x,df_y, cv=5, scoring="neg_mean_squared_error"
)
rf_rmse_scores=np.sqrt(-scores_rf)
print('RF Scores:', rf_rmse_scores)
print('RF Mean Scores:', rf_rmse_scores.mean())
# the random forest model looks promising

RF Scores: [59.41276261 55.62618795 46.5350657  41.26104856 63.1382542 ]
RF Mean Scores: 53.19466380331461


In [29]:
# hyperparameter tuning
from sklearn.model_selection import GridSearchCV
para_grid = [{'n_estimators': [20,30,40,50],'max_features':[5,10,20]}
    ]
forest_reg= RandomForestRegressor()

rfgrid=GridSearchCV(forest_reg,para_grid,cv=5,scoring="neg_mean_squared_error",return_train_score=True)

In [30]:
rfgrid.fit(df_x,df_y)
print(rfgrid.best_params_, rfgrid.best_score_)


{'max_features': 10, 'n_estimators': 50} -3198.373835991908


In [31]:
cvres=rfgrid.cv_results_
for mean_score, params in zip(cvres['mean_test_score'],cvres['params']):
    print(np.sqrt(-mean_score),params)

57.58121387101108 {'max_features': 5, 'n_estimators': 20}
57.522688097484604 {'max_features': 5, 'n_estimators': 30}
57.84446722196813 {'max_features': 5, 'n_estimators': 40}
57.083787562556914 {'max_features': 5, 'n_estimators': 50}
57.97074171555614 {'max_features': 10, 'n_estimators': 20}
60.13923833649437 {'max_features': 10, 'n_estimators': 30}
57.904638800478814 {'max_features': 10, 'n_estimators': 40}
56.55416727343714 {'max_features': 10, 'n_estimators': 50}
60.26210112684283 {'max_features': 20, 'n_estimators': 20}
59.50063745397062 {'max_features': 20, 'n_estimators': 30}
57.41408539367428 {'max_features': 20, 'n_estimators': 40}
56.90577087829801 {'max_features': 20, 'n_estimators': 50}


# error analysis

In [10]:
df['eui_lasso_prediction']=pd.DataFrame(eui_lasso_prediction)

In [11]:
df['lasso_prediction_error']=abs(df.site_eui-df.eui_lasso_prediction)

In [12]:
print(min(df['lasso_prediction_error']))
print(max(df['lasso_prediction_error']))
print(df['lasso_prediction_error'].mean())

6.72505970982229e-05
912.4661372546848
27.670350170493577


In [13]:
df_big_error=df[df.lasso_prediction_error>100]
df_small_error=df[df.lasso_prediction_error<50]

In [14]:
df_big_error.describe()

,floor_area,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,...,facility_type_Education_Other_classroom,facility_type_Lodging_Hotel,facility_type_Multifamily_Uncategorized,facility_type_Office_Uncategorized,facility_type_Retail_Uncategorized,facility_type_Warehouse_Nonrefrigerated,facility_type_other,site_eui,eui_lasso_prediction,lasso_prediction_error
count,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,...,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000,2317.000000
mean,0.139730,-0.486531,0.128624,-0.003715,0.006374,-0.036614,0.085289,0.047164,0.117523,-0.005123,...,0.004316,0.027190,0.164868,0.098835,0.023738,0.004748,0.544238,301.536764,99.231937,208.471978
std,1.476892,1.152992,1.070486,1.180371,1.148330,1.176562,1.091354,1.100650,0.983341,1.137203,...,0.065568,0.162673,0.371142,0.298504,0.152263,0.068753,0.498147,147.242932,26.189464,137.428011
min,-0.668522,-2.717372,-0.621640,-3.244052,-3.359607,-3.184614,-1.965508,-2.512433,-2.078099,-3.059332,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.467170,30.451129,100.017551
25%,-0.449556,-1.340329,-0.455128,-0.792279,-0.722178,-0.570441,-0.772873,-0.440058,-0.295474,-0.860264,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,217.100000,80.438957,122.899444
50%,-0.325153,0.165812,-0.239156,-0.046087,0.020175,-0.010261,0.022217,-0.160116,0.298735,0.339228,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,260.879853,94.551377,158.330488
75%,0.126008,0.165812,0.052652,0.167110,0.430544,0.549919,0.658290,0.603664,0.655260,0.539143,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,335.879781,117.793858,230.986620
max,22.152606,1.585887,11.408458,3.578273,4.282942,5.591538,2.407488,2.682359,2.794410,2.538296,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,997.866120,197.609877,912.466137


In [15]:
df_small_error.describe()

,floor_area,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,...,facility_type_Education_Other_classroom,facility_type_Lodging_Hotel,facility_type_Multifamily_Uncategorized,facility_type_Office_Uncategorized,facility_type_Retail_Uncategorized,facility_type_Warehouse_Nonrefrigerated,facility_type_other,site_eui,eui_lasso_prediction,lasso_prediction_error
count,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,...,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000,65746.000000
mean,-0.005196,0.055741,-0.009287,-0.007320,-0.009198,-0.011059,-0.020860,-0.011142,-0.015359,-0.008095,...,0.055562,0.028275,0.557616,0.179479,0.014769,0.011681,0.078484,75.818269,80.743189,16.719017
std,0.957464,0.974640,0.971671,0.984366,0.984444,0.977420,0.992925,0.991818,1.000624,0.988297,...,0.229076,0.165760,0.496673,0.383755,0.120628,0.107448,0.268934,29.537620,22.271804,12.672961
min,-0.663431,-2.674340,-0.756828,-3.244052,-3.359607,-3.184614,-1.965508,-2.512433,-2.434624,-3.059332,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.090174,25.328676,0.000067
25%,-0.417151,-0.264514,-0.455128,-0.365884,-0.640772,-0.570441,-0.772873,-0.440058,-0.414315,-0.960221,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,57.324399,67.119614,6.350231
50%,-0.299812,0.165812,-0.239156,-0.046087,0.020175,-0.010261,-0.216310,-0.160116,0.298735,0.339228,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,75.207241,79.410484,13.700367
75%,-0.003013,0.682203,0.052652,0.167110,0.430544,0.549919,0.658290,0.603664,0.417577,0.539143,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,93.743169,91.288189,24.771617
max,25.063020,1.585887,31.076688,4.004668,4.352105,5.964991,2.884542,3.336135,3.626302,3.038085,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,208.824795,163.801486,49.987780


In [16]:
df_big_error.head()

,floor_area,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,...,facility_type_Education_Other_classroom,facility_type_Lodging_Hotel,facility_type_Multifamily_Uncategorized,facility_type_Office_Uncategorized,facility_type_Retail_Uncategorized,facility_type_Warehouse_Nonrefrigerated,facility_type_other,site_eui,eui_lasso_prediction,lasso_prediction_error
0,-0.424272,-2.244014,-0.611748,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,248.682615,125.164179,123.518437
13,1.826069,-2.157949,-0.611748,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,608.839519,129.064964,479.774556
24,-0.399547,-1.598525,-0.621640,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,287.863448,112.470657,175.392791
25,-0.169268,-0.823938,-0.621640,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,241.932986,97.650186,144.282800
26,-0.387501,-0.350580,-0.621640,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,264.068722,87.697823,176.370899


In [17]:
df_small_error.head()

,floor_area,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,...,facility_type_Education_Other_classroom,facility_type_Lodging_Hotel,facility_type_Multifamily_Uncategorized,facility_type_Office_Uncategorized,facility_type_Retail_Uncategorized,facility_type_Warehouse_Nonrefrigerated,facility_type_other,site_eui,eui_lasso_prediction,lasso_prediction_error
2,0.461940,1.456789,-0.621640,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,24.693619,53.898796,29.205177
3,-0.448240,-0.737873,-0.621640,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,48.406926,80.416982,32.010057
4,-0.404999,1.585887,-0.611748,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.899395,34.263557,30.364162
7,-0.468100,1.542855,-0.611748,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,22.392014,34.961023,12.569008
8,-0.178082,1.499822,-0.611748,2.618884,2.314092,1.670279,1.850925,1.698778,1.724835,1.838593,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,28.199578,36.539547,8.339970


In [18]:
print(df_big_error.columns)
print(df_big_error['facility_type_other'].value_counts())
print(df_small_error['facility_type_other'].value_counts())
print(df_big_error['building_class_Commercial'].value_counts())
print(df_big_error['building_class_Residential'].value_counts())
# prediction is not good for commercial and facility type other

Index(['floor_area', 'energy_star_rating', 'ELEVATION', 'january_min_temp',
       'january_avg_temp', 'january_max_temp', 'february_min_temp',
       'february_avg_temp', 'february_max_temp', 'march_min_temp',
       'march_avg_temp', 'march_max_temp', 'april_min_temp', 'april_avg_temp',
       'april_max_temp', 'may_min_temp', 'may_avg_temp', 'may_max_temp',
       'june_min_temp', 'june_avg_temp', 'june_max_temp', 'july_min_temp',
       'july_avg_temp', 'july_max_temp', 'august_min_temp', 'august_avg_temp',
       'august_max_temp', 'september_min_temp', 'september_avg_temp',
       'september_max_temp', 'october_min_temp', 'october_avg_temp',
       'october_max_temp', 'november_min_temp', 'november_avg_temp',
       'november_max_temp', 'december_min_temp', 'december_avg_temp',
       'december_max_temp', 'cooling_degree_days', 'heating_degree_days',
       'precipitation_inches', 'snowfall_inches', 'snowdepth_inches',
       'avg_temp', 'days_below_30F', 'days_below_20F', 'days_

In [19]:
# Note in the ver01
# Lasso model(alpha=1) is underfitting
# Tree model is overfitting
#error analysis for lasso implies that 
#'facility_type_other' is not well predicted 
# To do list
# increase the category of facility 
# feature engineering(outlier, nonlinear relationship)
# Randome Forest